## Extracting datat from xnli and rte3, cleaning and combining them in on big dataset

In [1]:
import pandas as pd 
import os
import xml

data_path = "data/"
rte3_path = data_path + "rte3/"
xnli_path = data_path + "xnli/"

preprocessed_path = data_path + "preprocessed/"
rte3_preprocessed_path = os.path.join(preprocessed_path, "rte3.tsv")
xnli_preprocessed_path = os.path.join(preprocessed_path, "xnli.tsv")

### Process RTE3

In [2]:
columns=["sentenceA", "sentenceB", "label"]
dataframe = pd.DataFrame(columns=columns)

import xml.etree.ElementTree as ET
devel_file = os.path.join(rte3_path, "german_social_media_DEVEL.xml")
test_file = os.path.join(rte3_path, "german_social_media_TEST.xml")


def parse_and_collect(filepath, rte3_dataframe):
    dom = ET.parse(filepath)
    for pair in dom.iter("pair"):
        if pair.tag == "pair":
            label = pair.attrib["entailment"]
            sentenceA = pair[0].text
            sentenceB = pair[1].text

            rte3_dataframe = rte3_dataframe.append({"sentenceA": sentenceA, "sentenceB": sentenceB, "label": label.lower().replace("nonentailment","contradiction")}, ignore_index=True)
    return rte3_dataframe

dataframe = parse_and_collect(devel_file, dataframe)
dataframe = parse_and_collect(test_file, dataframe)
dataframe.to_csv(os.path.join(data_path + "preprocessed", "rte3.tsv"),sep="\t", encoding="utf8", index=False)

dataframe.head(), len(dataframe)

(                                           sentenceA  \
 0  Hallo Folgendes Problem, der CPU ist bei 100%....   
 1  Hallo, habe mir ein Asus P5Q geholt und am Anf...   
 2  hallo liebe mods, ich schreibe diesen beitrag ...   
 3  Hallo, ich habe Probleme mit dem Trojaner '' T...   
 4  Hallo, ich habe Probleme mit dem Trojaner '' T...   
 
                                            sentenceB       label  
 0  Der Computer ist durch Malware verseucht, der ...  entailment  
 1  Die USB-Anschlüsse funktionieren bei dem neuen...  entailment  
 2  Beim Aufrufen des p5q deluxe Sammelthreads bek...  entailment  
 3            Ein Trojaner kann nicht entfernt werden  entailment  
 4  Mein Virenprogramm AntiVir meldet immer wieder...  entailment  ,
 3014)

### Process XNLI

In [3]:
columns=["sentenceA", "sentenceB", "label"]
dataframe = pd.DataFrame(columns=columns)

devel_file = os.path.join(xnli_path, "xnli.dev.tsv")
test_file = os.path.join(xnli_path, "xnli.test.tsv")

def parse_and_collect(filepath, xnli_dataframe):
    xnli_data = pd.read_csv(filepath, encoding="utf8", sep='\t')
    for idx, row in xnli_data.iterrows():
        if row["language"] == "de":
            xnli_dataframe = xnli_dataframe.append({"sentenceA": row["sentence1"], "sentenceB": row["sentence2"], "label": row["gold_label"].lower()}, ignore_index=True)
    return xnli_dataframe
    
dataframe = parse_and_collect(devel_file, dataframe)
dataframe = parse_and_collect(test_file, dataframe)
dataframe.to_csv(os.path.join(data_path + "preprocessed", "xnli.tsv"), sep="\t", encoding="utf8", index=False)

dataframe.head(), len(dataframe)

(                                           sentenceA  \
 0            und er hat gesagt, Mama ich bin daheim.   
 1            und er hat gesagt, Mama ich bin daheim.   
 2            und er hat gesagt, Mama ich bin daheim.   
 3  Ich wusste nicht was ich vorhatte oder so, ich...   
 4  Ich wusste nicht was ich vorhatte oder so, ich...   
 
                                            sentenceB          label  
 0  Er rief seine Mutter an, sobald er aus dem Sch...        neutral  
 1                                Er sagte kein Wort.  contradiction  
 2  Er sagte seiner Mutter, er sei nach Hause geko...     entailment  
 3  Ich war noch nie in Washington, deshalb habe i...        neutral  
 4  Ich wusste genau, was ich tun musste, als ich ...  contradiction  ,
 7500)

### Combin both and build sets

In [4]:
xnli_dataframe = pd.read_csv(rte3_preprocessed_path, encoding="utf8", sep='\t')
rte3_dataframe = pd.read_csv(xnli_preprocessed_path, encoding="utf8", sep='\t')

all_data = pd.concat([xnli_dataframe, rte3_dataframe])
all_data = all_data.sample(frac=1)

train_size=0.8
train_set = all_data.sample(frac=train_size, random_state=0)
test_set = all_data.drop(train_set.index)

train_set.to_csv(os.path.join(preprocessed_path, "train.tsv"), sep="\t", encoding="utf8", index=False)
test_set.to_csv(os.path.join(preprocessed_path, "test.tsv"), sep="\t", encoding="utf8", index=False)